## Import Libraries

In [2]:
import nltk
import pandas as pd
import string


## Load csv files to data frames


In [34]:
train_df = pd.read_csv('data/train.csv',encoding = "latin1")
test_df = pd.read_csv('data/test.csv',encoding = "latin1")
attributes_df = pd.read_csv('data/attributes.csv',encoding = "latin1")
attributes_df[["product_uid"]] = attributes_df[["product_uid"]].astype(int)
product_descriptions_df = pd.read_csv('data/product_descriptions.csv',encoding = "latin1")
sample_submission_df = pd.read_csv('data/sample_submission.csv', encoding = "latin1")


ValueError: Cannot convert NA to integer

In [33]:
print(attributes_df.dtypes)

product_uid    float64
name            object
value           object
dtype: object


In [7]:
print(sample_submission_df.head())

   id  relevance
0   1          1
1   4          1
2   5          1
3   6          1
4   7          1


# Data Cleaning

Download NLTK's stopwords list and WordNetLemmatizer

In [8]:
lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()
stopwords=nltk.corpus.stopwords.words('english')
# nltk.download('stopwords')
# nltk.download('wordnet')


Process and tokenize the raw text by:
    1. Convert to lower case
    2. Remove apostrophe
    3. Remove Punctuation
    4. Lemmatize

In [9]:
def process(text, lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()):
   
    # convert text to lower case
    lowercase = str(text).lower()
    
    #remove 's from string
    apoRemoved = lowercase.replace("'s","")
    
    #convert don't to dont
    apoRemoved = apoRemoved.replace("'","")
    
    #handle other punctuations
    transtable = str.maketrans(string.punctuation,"                                ")
    brokenWords = apoRemoved.translate(transtable)
    
    #convert string to list of words
    listOfWords =  nltk.word_tokenize(brokenWords)
    
    #lemmatize text
    lemmatizedList=[lemmatizer.lemmatize(word) for word in listOfWords]
   
    return lemmatizedList


In [12]:
# test case
text = "Dogs Here's don't cars."
print(process(text))

['dog', 'here', 'dont', 'car']


In [14]:
# process train_df
def process_train_df(df, lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()):
    
    newdf=df
    for i,row in newdf.iterrows():
#         if i>5:
#             break
#         print("raw: ",text)
        newdf.at[i,'product_title'] = process(row['product_title'])
#         print("processed: ",df.iloc[i]['product_title'])
    return newdf


In [15]:
processed_train_df = process_train_df(train_df)
print(processed_train_df.head())

   id  product_uid                                      product_title  \
0   2       100001             [impson, trong, tie, 12, gauge, angle]   
1   3       100001             [impson, trong, tie, 12, gauge, angle]   
2   9       100002  [behr, premium, textured, deckover, 1, gal, c,...   
3  16       100005  [delta, vero, 1, handle, hower, only, faucet, ...   
4  17       100005  [delta, vero, 1, handle, hower, only, faucet, ...   

          search_term  relevance  
0       angle bracket       3.00  
1           l bracket       2.50  
2           deck over       3.00  
3    rain shower head       2.33  
4  shower only faucet       2.67  


In [ ]:
# process 
def process_train_df(df, lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()):
    
    newdf=df
    for i,row in newdf.iterrows():
#         if i>5:
#             break
#         print("raw: ",text)
        newdf.at[i,'product_title'] = process(row['product_title'])
#         print("processed: ",df.iloc[i]['product_title'])
    return newdf